<a href="https://colab.research.google.com/github/banakonwee/.github/blob/main/5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kivy

In [ ]:
from kivy.lang import Builder
from kivy.uix.tabbedpanel import TabbedPanel
from kivy.core.window import Window
from kivy.app import App
from kivy.uix.popup import Popup
from kivy.uix.label import Label
import sqlite3
import os

DB_NAME = "credit_management.db"

Builder.load_string('''
<CreditManagementSystem>:
    size_hint: 1, 1
    pos_hint: {'center_x': 0.5, 'center_y': 0.5}
    do_default_tab: False

    TabbedPanelItem:
        text: 'Customers'
        BoxLayout:
            orientation: 'vertical'
            padding: 10
            spacing: 10

            BoxLayout:
                size_hint_y: None
                height: 30
                Label:
                    text: 'Name:'
                TextInput:
                    id: name_input
            BoxLayout:
                size_hint_y: None
                height: 30
                Label:
                    text: 'Email:'
                TextInput:
                    id: email_input
            BoxLayout:
                size_hint_y: None
                height: 30
                Label:
                    text: 'Phone:'
                TextInput:
                    id: phone_input
            BoxLayout:
                size_hint_y: None
                height: 30
                Label:
                    text: 'Address:'
                TextInput:
                    id: address_input
            BoxLayout:
                size_hint_y: None
                height: 40
                spacing: 5
                Button:
                    text: 'Add Customer'
                    on_press: root.add_customer()
                Button:
                    text: 'Clear'
                    on_press: root.clear_customer_fields()

            RecycleView:
                viewclass: 'Label'
                data: [{'text': str(x)} for x in root.customers_data]
                RecycleBoxLayout:
                    size_hint_y: None
                    height: self.minimum_height
                    orientation: 'vertical'

    TabbedPanelItem:
        text: 'Loans'
        BoxLayout:
            orientation: 'vertical'
            padding: 10
            spacing: 10

            BoxLayout:
                size_hint_y: None
                height: 30
                Label:
                    text: 'Customer ID:'
                TextInput:
                    id: customer_id_input
            BoxLayout:
                size_hint_y: None
                height: 30
                Label:
                    text: 'Loan Amount:'
                TextInput:
                    id: loan_amount_input
            BoxLayout:
                size_hint_y: None
                height: 30
                Label:
                    text: 'Loan Term:'
                TextInput:
                    id: loan_term_input
            BoxLayout:
                size_hint_y: None
                height: 30
                Label:
                    text: 'Income:'
                TextInput:
                    id: income_input
            BoxLayout:
                size_hint_y: None
                height: 40
                spacing: 5
                Button:
                    text: 'Add Loan'
                    on_press: root.add_loan()
                Button:
                    text: 'Clear'
                    on_press: root.clear_loan_fields()

            RecycleView:
                viewclass: 'Label'
                data: [{'text': str(x)} for x in root.loans_data]
                RecycleBoxLayout:
                    size_hint_y: None
                    height: self.minimum_height
                    orientation: 'vertical'
''')

class CreditManagementSystem(TabbedPanel):
    customers_data = []
    loans_data = []

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.create_database()
        Window.bind(on_keyboard=self.keyboard_handler)
        self.load_customers()
        self.load_loans()

    def create_database(self):
        """Create database tables if not exist"""
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Customers (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                email TEXT,
                phone TEXT,
                address TEXT
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Loans (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                customer_id INTEGER,
                loan_amount REAL,
                loan_term INTEGER,
                income REAL,
                FOREIGN KEY(customer_id) REFERENCES Customers(id)
            )
        ''')
        conn.commit()
        conn.close()

    def add_customer(self):
        name = self.ids.name_input.text
        email = self.ids.email_input.text
        phone = self.ids.phone_input.text
        address = self.ids.address_input.text

        if not all([name, email, phone, address]):
            self.show_popup("Error", "All fields are required!")
            return

        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        cursor.execute("INSERT INTO Customers (name, email, phone, address) VALUES (?, ?, ?, ?)",
                       (name, email, phone, address))
        conn.commit()
        conn.close()
        self.load_customers()
        self.clear_customer_fields()

    def load_customers(self):
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM Customers")
        self.customers_data = cursor.fetchall()
        conn.close()

    def clear_customer_fields(self):
        self.ids.name_input.text = ""
        self.ids.email_input.text = ""
        self.ids.phone_input.text = ""
        self.ids.address_input.text = ""

    def add_loan(self):
        customer_id = self.ids.customer_id_input.text
        loan_amount = self.ids.loan_amount_input.text
        loan_term = self.ids.loan_term_input.text
        income = self.ids.income_input.text

        if not all([customer_id, loan_amount, loan_term, income]):
            self.show_popup("Error", "All fields are required!")
            return

        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        cursor.execute("INSERT INTO Loans (customer_id, loan_amount, loan_term, income) VALUES (?, ?, ?, ?)",
                       (customer_id, loan_amount, loan_term, income))
        conn.commit()
        conn.close()
        self.load_loans()
        self.clear_loan_fields()

    def load_loans(self):
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM Loans")
        self.loans_data = cursor.fetchall()
        conn.close()

    def clear_loan_fields(self):
        self.ids.customer_id_input.text = ""
        self.ids.loan_amount_input.text = ""
        self.ids.loan_term_input.text = ""
        self.ids.income_input.text = ""

    def show_popup(self, title, message):
        content = Label(text=message)
        popup = Popup(title=title, content=content, size_hint=(None, None), size=(400, 200))
        popup.open()

    def keyboard_handler(self, instance, key, *args):
        if key == 27:  # Android back button
            return True

class CreditApp(App):
    Kv_file = None # Changed 'none' to 'None'
    def build(self):
        return CreditManagementSystem()

In [ ]:
!pip install buildozer

In [ ]:
[app]

# Title of your application
title = "Credit Management"

# Package name
package.name = creditapp

# Package domain (needed for android/ios packaging)
package.domain = org.example

# Source code where the main.py is
source.dir = "."

# Source files to include (let empty to include all the files)
source.include_exts = py,png,jpg,kv,db

# Main entry point
main.py = main.py

# Android specific
# Requirements for the build
requirements = kivy==2.0

# Android permissions
android.permissions = WRITE_EXTERNAL_STORAGE, READ_EXTERNAL_STORAGE

# Presplash
presplash.filename = credit_management.db.dir/presplash.png

# Icon
icon.filename = credit_managment.db/icon.png

# Orientation (portrait)
orientation = portrait

# Android API level
android.api = 30

# Minimum Android SDK version
android.minapi = 21

# Android SDK directory (if empty, buildozer will try to auto-detect it)
# android.sdk_path =

# Android NDK directory (if empty, buildozer will try to auto-detect it)
# android.ndk_path =

# Android entry point
android.entrypoint = org.kivy.android.PythonActivity

# Android app theme
android.apptheme = "@android:style/Theme.NoTitleBar"

# Android logcat filters (default)
# android.logcat_filters = *:S python:D

# (bool) Copy SQLite3 database
android.copy_libs = 1

In [ ]:
[app]

# Title of your application
title = "Credit Management"

# Package name
package.name = creditapp

# Package domain (needed for android/ios packaging)
package.domain = org.example

# Source code where the main.py is
source.dir = "credit management.db" to ".", indicating the current directory

# Source files to include (let empty to include all the files)
source.include_exts = py,png,jpg,kv,db

# Main entry point
main.py = main.py # Assuming your main Python file is named main.py

# Android specific
# Requirements for the build
requirements = kivy==2.0.0, sqlite3 # Added sqlite3

# Android permissions
android.permissions = WRITE_EXTERNAL_STORAGE, READ_EXTERNAL_STORAGE

# Presplash (Make sure you have a presplash.png in your project directory)
# presplash.filename = presplash.png

# Icon (Make sure you have an icon.png in your project directory)
# icon.filename = icon.png

# Orientation (portrait)
orientation = portrait

# Android API level
android.api = 30

# Minimum Android SDK version
android.minapi = 21

In [ ]:
!buildozer init

In [ ]:
!pip cython

In [ ]:
!buildozer android debug

In [ ]:
[app]
title = "Credit Management"
[app]
title = "Credit Management"
[app]
title = "Credit Management"
[app]
title = "Credit Management"
package.name = creditapp
package.domain = org.example
[app]
title = "Credit Management"
package.name = creditapp
package.domain = org.example
source.dir = none
source.include_exts = py,png,jpg,kv,db
requirements = kivy==2
android.permissions = WRITE_EXTERNAL_STORAGE,READ_EXTERNAL_STORAGE
android.permissions = WRITE_EXTERNAL_STORAGE,READ_EXTERNAL_STORAGE
android.permissions = WRITE_EXTERNAL_STORAGE, READ_EXTERNAL_STORAGE

In [ ]:
[app]

# (str) Title of your application
title = "Credit Management"

# (str) Package name
package.name = creditapp

# (str) Package domain (needed for android/ios packaging)
package.domain = org.example

# (str) Source code where the main.py liv
source.include_exts = py,png,jpg,kv
# (list) Source files to include (let empty to include all the files)
source.include_exts = py,png,jpg,kv,db

# (str) Application versioning (method 1)
# version = 0.1
# version.regex = __version__ = ['"](.*)['"]
# version.filename = %(source.dir)s/main.py

# (list) Application requirements
# comma separated e.g. requirements = sqlite3,kivy
requirements = python3,kivy==2


# (str) Custom source folders for requirements (optional)
# Sets custom source for any requirements with subdirectory (optional)
# source.requirements.dir = ../../requirements

# (list) Garden requirements
# garden_requirements =

# (str) Presplash of the application
#presplash.filename = %(source.dir)s/data/presplash.png

# (str) Icon of the application
#icon.filename = %(source.dir)s/data/icon.png

# (str) Supported orientation (one of landscape, sensorLandscape, sensorPortrait or portrait)
#
# Landscape or portrait can be forced by adding "-c CONFIG" to the buildozer command line.
orientation = portrait

# (list) Permissions
android.permissions = WRITE_EXTERNAL_STORAGE, READ_EXTERNAL_STORAGE
# (int) Target Android API, should be as high as possible.
# android.api = 27

# (int) Minimum API required on devices (uses android.api if not set)
# This should be the same as the required API of the project you build.
android.minapi = 21

# (list) Android color themes, default "None"
# android.themes = ["Theme.Light.NoTitleBar"]

# (list) List of Java .jar files to add to the libs so they can be packaged in the APK
# android.add_jars = ["src/main/java/libs/mylib.jar"]

# (string) Java entry point, for java-based app only. default is based on the
# package name.
# android.entrypoint = org.kivy.android.PythonActivity

# (list) List of Java .aar archives to add (currently works only with sdl2_gradle
# bootstrap)
# android.add_aars = ["src/main/libs/mylib.aar"]

# (list) List of Java files to add to the android project (can be java or a
# directory containing the files)
# android.add_src = ["src/main/java/extra.java"]

# (list) List of assets that will be copied to the assets folder
# android.add_assets = ["assets/*"]

# (list) List of folders that will be copied to the assets/lib folder
android.add_libs_armeabi = ["libs/armeabi", "libs/armeabi-v7a"]
android.add_libs_armeabi_v7a = ["libs/armeabi-v7a"]
android.add_libs_arm64_v8a = ["libs/arm64-v8a"]
android.add_libs_x86_64 = ["libs/x86_64"]

# (bool) Indicate if the .apk should be signed. If set to False, no signing is
# done and you can sign the file manually after building (with certtool
# or jarsigner).
# android.sign_apk = True

# (str) Path to the keystore only needed if sign_apk is True
# android.keystore = ~/.android/debug.keystore

# (str) Alias name to the keystore key only needed if sign_apk is True
# android.keyalias = androiddebugkey

# (str) Keystore password only needed if sign_apk is True
# android.keystore_pass = android

# (str) Key alias password only needed if sign_apk is True
# android.keyalias_pass = android

# (list) Android entry point, for java-based app only.
# It is usually better to use the list than a string,
# to avoid the risk of having a package name with dots.
# "org.example.myapp" becomes "['org', 'example', 'myapp']"
# android.entrypoint = ['org', 'kivy', 'android', 'PythonActivity']

# (list) Pattern to whitelist for the whole project
# android.whitelist = []
# (str) Path to a custom whitelist template
# android.whitelist_src = ""

# (str) Path to a custom blacklist file
# android.blacklist_src = ""

# (list) List of Java .jar files to exclude from packaging in the APK
# android.exclude_jars = ["mylib.jar"]

# (list) List of Java .aar archives to exclude from packaging in the APK
# android.exclude_aars = ["mylib.aar"]

# (list) List of Java files to exclude from packaging in the APK
# android.exclude_src = ["extra.java"]

# (string) Path to the N

In [ ]:
[app]
[app]

# (str) Title of your application
title = Credit Management

# (str) Package name
package.name = creditapp

# (str) Package domain (needed for android/ios packaging)
package.domain = org.example

# (str) Source code where the main.py lives
source.dir = .

# (list) Source files to include (let empty to include all the files)
source.include_exts = py,png,jpg,kv,db

# (str) Application versioning (method 1)
# version = 0.1
# version.regex = __version__ = ['"](.*)['"]
# version.filename = %(source.dir)s/main.py

# (list) Application requirements
# comma separated e.g. requirements = sqlite3,kivy
requirements = python3,kivy==2.0.0,sqlite3

# (str) Custom source folders for requirements (optional)
# Sets custom source for any requirements with subdirectory (optional)
# source.requirements.dir = ../../requirements

# (list) Garden requirements
# garden_requirements =

# (str) Presplash of the application
#presplash.filename = %(source.dir)s/data/presplash.png

# (str) Icon of the application
#icon.filename = %(source.dir)s/data/icon.png

# (str) Supported orientation (one of landscape, sensorLandscape, sensorPortrait or portrait)
#
# Landscape or portrait can be forced by adding "-c CONFIG" to the buildozer command line.
orientation = portrait

# (list) Permissions
android.permissions = WRITE_EXTERNAL_STORAGE,READ_EXTERNAL_STORAGE
# (int) Target Android API, should be as high as possible.
# android.api = 27

# (int) Minimum API required on devices (uses android.api if not set)
# This should be the same as the required API of the project you build.
android.minapi = 21

# (list) Android color themes, default "None"
# android.themes = ["Theme.Light.NoTitleBar"]

# (list) List of Java .jar files to add to the libs so they can be packaged in the APK
# android.add_jars = ["src/main/java/libs/mylib.jar"]

# (string) Java entry point, for java-based app only. default is based on the
# package name.
# android.entrypoint = org.kivy.android.PythonActivity

# (list) List of Java .aar archives to add (currently works only with sdl2_gradle
# bootstrap)
# android.add_aars = ["src/main/libs/mylib.aar"]

# (list) List of Java files to add to the android project (can be java or a
# directory containing the files)
# android.add_src = ["src/main/java/extra.java"]

# (list) List of assets that will be copied to the assets folder
# android.add_assets = ["assets/*"]

# (list) List of folders that will be copied to the assets/lib folder
#android.add_libs_armeabi = ["libs/armeabi", "libs/armeabi-v7a"]
#android.add_libs_armeabi_v7a = ["libs/armeabi-v7a"]
#android.add_libs_arm64_v8a = ["libs/arm64-v8a"]
#android.add_libs_x86_64 = ["libs/x86_64"]

# (bool) Indicate if the .apk should be signed. If set to False, no signing is
# done and you can sign the file manually after building (with certtool
# or jarsigner).
# android.sign_apk = True

# (str) Path to the keystore only needed if sign_apk is True
# android.keystore = ~/.android/debug.keystore

# (str) Alias name to the keystore key only needed if sign_apk is True
# android.keyalias = androiddebugkey

# (str) Keystore password only needed if sign_apk is True
# android.keystore_pass = android

# (str) Key alias password only needed if sign_apk is True
# android.keyalias_pass = android

# (list) Android entry point, for java-based app only.
# It is usually better to use the list than a string,
# to avoid the risk of having a package name with dots.
# "org.example.myapp" becomes "['org', 'example', 'myapp']"
# android.entrypoint = ['org', 'kivy', 'android', 'PythonActivity']

# (list) Pattern to whitelist for the whole project
# android.whitelist = []
# (str) Path to a custom whitelist template
# android.whitelist_src = ""

# (str) Path to a custom blacklist file
# android.blacklist_src = ""

# (list) List of Java .jar files to exclude from packaging in the APK
# android.exclude_jars = ["mylib.jar"]

# (list) List of Java .aar archives to exclude from packaging in the APK
# android.exclude_aars = ["mylib.aar"]

# (list) List of Java files to exclude from packaging in the APK
# android.exclude_src = ["extra.java"]

# (string) Path to the NDK directory (if set, make sure the NDK version matches
# android.ndk_version)
# android.ndk_path =

# (string) Path to the SDK directory (if set, make sure the NDK version matches
# android.ndk_version)
# android.sdk_path =

# (string) NDK API to use. This is the minimum API your app will support,
# it should usually match android.minapi.
# android.ndk_api =

# (string) NDK version to use.
# android.ndk_version =

# (string) The filename of the androidmanifest.xml template file to use to build
# the manifest
# android.manifest_template =

# (bool) Use --private data storage (True) or --dir public storage (False)
# android.private_storage = True

# (str) Android NDK directory (if set, make sure the NDK version matches
# android.ndk_version)
# android.ndk_path =

# (str) Android SDK directory (if set, make sure the NDK version matches
# android.ndk_version)
# android.sdk_path =

# (str) ANT directory (if set, make sure the NDK version matches
# android.ndk_version)
# android.ant_path =
package.name = creditapp

# (str) Package domain (needed for android/ios packaging)
package.domain = org.example

# (str) Source code where the main.py lives
source.dir = .

# (list) Source files to include (let empty to include all the files)
source.include_exts = py,png,jpg,kv,db

# (str) Application versioning (method 1)
# version = 0.1
# version.regex = __version__ = ['"](.*)['"]
# version.filename = %(source.dir)s/main.py

# (list) Application requirements
# comma separated e.g. requirements = sqlite3,kivy
requirements = python3,kivy==2.0.0, sqlite3

# (str) Custom source folders for requirements (optional)
# Sets custom source for any requirements with subdirectory (optional)
# source.requirements.dir = ../../requirements

# (list) Garden requirements
# garden_requirements =

# (str) Presplash of the application
#presplash.filename = %(source.dir)s/data/presplash.png

# (str) Icon of the application
#icon.filename = %(source.dir)s/data/icon.png

# (str) Supported orientation (one of landscape, sensorLandscape, sensorPortrait or portrait)
#
# Landscape or portrait can be forced by adding "-c CONFIG" to the buildozer command line.
orientation = portrait

# (list) Permissions
android.permissions = WRITE_EXTERNAL_STORAGE, READ_EXTERNAL_STORAGE
# (int) Target Android API, should be as high as possible.
# android.api = 27

# (int) Minimum API required on devices (uses android.api if not set)
# This should be the same as the required API of the project you build.
android.minapi = 21

# (list) Android color themes, default "None"
# android.themes = ["Theme.Light.NoTitleBar"]

# (list) List of Java .jar files to add to the libs so they can be packaged in the APK
# android.add_jars = ["src/main/java/libs/mylib.jar"]

# (string) Java entry point, for java-based app only. default is based on the
# package name.
# android.entrypoint = org.kivy.android.PythonActivity

# (list) List of Java .aar archives to add (currently works only with sdl2_gradle
# bootstrap)
# android.add_aars = ["src/main/libs/mylib.aar"]

# (list) List of Java files to add to the android project (can be java or a
# directory containing the files)
# android.add_src = ["src/main/java/extra.java"]

# (list) List of assets that will be copied to the assets folder
# android.add_assets = ["assets/*"]

# (list) List of folders that will be copied to the assets/lib folder
#android.add_libs_armeabi = ["libs/armeabi", "libs/armeabi-v7a"]
#android.add_libs_armeabi_v7a = ["libs/armeabi-v7a"]
#android.add_libs_arm64_v8a = ["libs/arm64-v8a"]
#android.add_libs_x86_64 = ["libs/x86_64"]

# (bool) Indicate if the .apk should be signed. If set to False, no signing is
# done and you can sign the file manually after building (with certtool
# or jarsigner).
# android.sign_apk = True

# (str) Path to the keystore only needed if sign_apk is True
# android.keystore = ~/.android/debug.keystore

# (str) Alias name to the keystore key only needed if sign_apk is True
# android.keyalias = androiddebugkey

# (str) Keystore password only needed if sign_apk is True
# android.keystore_pass = android

# (str) Key alias password only needed if sign_apk is True
# android.keyalias_pass = android

# (list) Android entry point, for java-based app only.
# It is usually better to use the list than a string,
# to avoid the risk of having a package name with dots.
# "org.example.myapp" becomes "['org', 'example', 'myapp']"
# android.entrypoint = ['org', 'kivy', 'android', 'PythonActivity']

# (list) Pattern to whitelist for the whole project
# android.whitelist = []
# (str) Path to a custom whitelist template
# android.whitelist_src = ""

# (str) Path to a custom blacklist file
# android.blacklist_src = ""

# (list) List of Java .jar files to exclude from packaging in the APK
# android.exclude_jars = ["mylib.jar"]

# (list) List of Java .aar archives to exclude from packaging in the APK
# android.exclude_aars = ["mylib.aar"]

# (list) List of Java files to exclude from packaging in the APK
# android.exclude_src = ["extra.java"]

# (string) Path to the NDK directory (if set, make sure the NDK version matches
# android.ndk_version)
# android.ndk_path =

# (string) Path to the SDK directory (if set, make sure the NDK version matches
# android.ndk_version)
# android.sdk_path =

# (string) NDK API to use. This is the minimum API your app will support,
# it should usually match android.minapi.
# android.ndk_api =

# (string) NDK version to use.
# android.ndk_version =

# (string) The filename of the androidmanifest.xml template file to use to build
# the manifest
# android.manifest_template =

# (bool) Use --private data storage (True) or --dir public storage (False)
# android.private_storage = True

# (str) Android NDK directory (if set, make sure the NDK version matches
# android.ndk_version)
# android.ndk_path =

# (str) Android SDK directory (if set, make sure the NDK version matches
# android.ndk_version)
# android.sdk_path =

# (str) ANT directory (if set, make sure the NDK version matches
# android.ndk_version)
# android.ant_path =

In [ ]:
[app]

# Title of your application
title = "Credit Management"

# Package name
package.name = creditapp

# Package domain (needed for android/ios packaging)
package.domain = org.example

# Source code where the main.py is
# Source files to include
source.include_exts = py,png,jpg,kv,db

# Main entry point
main.py = main.py
[app]

# Title of your application
title = "Credit Management"

# Package name
package.name = creditapp

# Package domain (needed for android/ios packaging)
package.domain = org.example

# Source code where the main.py is
# Source files to include
source.include_exts = py,png,jpg,kv,db

# Main entry point
main.py = main.py
[app]

# Title of your application
title = "Credit Management"

# Package name
package.name = creditapp

# Package domain (needed for android/ios packaging)
package.domain = org.example

# Source code where the main.py is
# Source files to include
source.include_exts = py,png,jpg,kv,db

# Main entry point
main.py = main.py
[app]

# Title of your application
title = "Credit Management"

# Package name
package.name = creditapp

# Package domain (needed for android/ios packaging)
package.domain = org.example

# Source code where the main.py is
# Source files to include
source.include_exts = py,png,jpg,kv,db

# Main entry point
main.py = main.py

# Android requirements
requirements = kivy==2.0.0,python3

# Android permissions
android.permissions = WRITE_EXTERNAL_STORAGE,READ_EXTERNAL_STORAGE

# Orientation
orientation = portrait

# Android API configuration
android.api = 30
android.minapi = 21
android.ndk_path = /path/to/your/ndk  # Update this if needed
android.sdk_path = /path/to/your/sdk  # Update this if needed

# Icon/presplash (create these files or remove lines)
icon.filename = icon.png
presplash.filename = presplash.png
# Package name
package.name = creditapp

# Package domain (needed for android/ios packaging)
package.domain = org.example

# Source code where the main.py is
# Source files to include
source.include_exts = py,png,jpg,kv,db

# Main entry point
main.py = main.py

# Android requirements
requirements = kivy==2.0.0,python3

# Android permissions
android.permissions = WRITE_EXTERNAL_STORAGE,READ_EXTERNAL_STORAGE

# Orientation
orientation = portrait

# Android API configuration
android.api = 30
android.minapi = 21
android.ndk_path = /path/to/your/ndk  # Update this if needed
android.sdk_path = /path/to/your/sdk  # Update this if needed

# Icon/presplash (create these files or remove lines)
icon.filename = icon.png
presplash.filename = presplash.png
android.permissions = WRITE_EXTERNAL_STORAGE,READ_EXTERNAL_STORAGE

# Orientation
orientation = portrait

# Android API configuration
android.api = 30
android.minapi = 21
android.ndk_path = /path/to/your/ndk  # Update this if needed
android.sdk_path = /path/to/your/sdk  # Update this if needed

# Icon/presplash (create these files or remove lines)
icon.filename = icon.png
presplash.filename = presplash.png
android.permissions = WRITE_EXTERNAL_STORAGE,READ_EXTERNAL_STORAGE

# Orientation
orientation = portrait

# Android API configuration
android.api = 30
android.minapi = 21
android.ndk_path = /path/to/your/ndk  # Update this if needed
android.sdk_path = /path/to/your/sdk  # Update this if needed

# Icon/presplash (create these files or remove lines)
icon.filename = icon.png
presplash.filename = presplash.png
android.permissions = WRITE_EXTERNAL_STORAGE, READ_EXTERNAL_STORAGE

# Orientation
orientation = portrait

# Android API configuration
android.api = 30
android.minapi = 21
android.ndk_path = /path/to/your/ndk  # Update this if needed
android.sdk_path = /path/to/your/sdk  # Update this if needed

# Icon/presplash (create these files or remove lines)
icon.filename = icon.png
presplash.filename = presplash.png